In [1]:


import gym
import numpy as np

In [2]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

In [3]:
# Create Taxi environment
env = gym.make("Taxi-v3")

In [4]:
from agents.ppo    import PPO
from agents.agent import Agent
from discriminators.eairl    import EAIRL
from utils.utils             import RunningMeanStd, Dict, make_transition

from configparser            import ConfigParser
from argparse                import ArgumentParser

import os
import gymnasium as gym
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torch
from torch.autograd import Variable

os.makedirs('./model_weights', exist_ok=True)

env = gym.make("Taxi-v3")


# Define arguments for configuration
action_dim = 1
state_dim = env.observation_space.n
state_dim=5
print(action_dim, state_dim, action_dim)
print()
# parser = ArgumentParser('parameters')



# parser.add_argument('--test', type=bool, default=False, help="True if test, False if train (default: False)")
# parser.add_argument('--render', type=bool, default=False, help="(default: False)")
# parser.add_argument('--epochs', type=int, default=1001, help='number of epochs, (default: 1001)')
# parser.add_argument("--agent", type=str, default = 'ppo', help = 'actor training algorithm(default: ppo)')
# parser.add_argument("--discriminator", type=str, default = 'eairl', help = 'discriminator training algorithm(default: gail)')
# parser.add_argument("--save_interval", type=int, default = 100, help = 'save interval')
# parser.add_argument("--print_interval", type=int, default = 1, help = 'print interval')
# parser.add_argument('--tensorboard', type=bool, default=True, help='use_tensorboard, (default: True)')

# args = parser.parse_args()
class Args:
    def __init__(self):
        self.test = False              # True if test, False if train
        self.render = False            # Whether to render
        self.epochs = 1001             # Number of epochs
        self.agent = 'ppo'             # Actor training algorithm
        self.discriminator = 'eairl'   # Discriminator training algorithm
        self.save_interval = 100       # Save interval
        self.print_interval = 1        # Print interval
        self.tensorboard = True 

parser = ConfigParser()
args=Args()

parser.read('config.ini')
agent_args = Dict(parser,args.agent)

demonstrations_location_args = Dict(parser,'demonstrations_location',True)
discriminator_args = Dict(parser,args.discriminator)

1 5 1



In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device='cpu'
if args.tensorboard:
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter()
else:
    writer = None


if args.discriminator == 'eairl':
    discriminator = EAIRL(writer, device, state_dim, action_dim, discriminator_args)
else:
    raise NotImplementedError
    
if args.agent == 'ppo':
    algorithm = PPO(device, state_dim, action_dim, agent_args)
else:
    raise NotImplementedError
agent = Agent(algorithm, writer, device, state_dim, action_dim, agent_args, demonstrations_location_args)
if device == 'cuda':
    agent = agent.cuda()
    discriminator = discriminator.cuda()

In [6]:
state_rms = RunningMeanStd(state_dim)
print(state_rms.mean)

0.0


In [7]:
def state_to_vector(state):
    # Taxi-v3 state information
    taxi_row, taxi_col, passenger_location, destination_location = env.unwrapped.decode(state)

    # Create a 5-dimensional vector representation
    vector = [
        taxi_row,               # 1. Taxi row position
        taxi_col,               # 2. Taxi column position
        passenger_location,     # 3. Passenger location (0-4: 0, 1, 2, 3 are specific positions, 4 is in the taxi)
        destination_location,   # 4. Destination location (0-3)
        int(passenger_location == 4)  # 5. Flag if the passenger is in the taxi (1 if true, 0 otherwise)
    ]
    return torch.Tensor(vector) 

In [8]:
five_dim_vector = state_to_vector(env.reset()[0])


In [9]:
five_dim_vector

tensor([2., 0., 2., 1., 0.])

In [10]:
score_lst = []
discriminator_score_lst = []
score = 0.0
done=False
terminated=False
discriminator_score = 0
if agent_args.on_policy == True:
    state_lst = []
    state,_ = (env.reset())
    print(state)
    state=(state_to_vector(state))
    for n_epi in range(args.epochs):
        print("epi",n_epi)
        for t in range(agent_args.traj_length):
            print(t,"traj")
            state_lst.append(state)
            # state = Variable(torch.from_numpy(state).type(dtype))
            # action, log_prob = agent.get_action(torch.from_numpy(state).float().unsqueeze(0).to(device))
            # action, log_prob = agent.get_action(torch.from_numpy(np.array(state)).float().unsqueeze(0).to(device))
            action,log_prob =agent.get_action(state)
            
            # action=((action.data.argmax())).unsqueeze(0)
            # print(action,log_prob,state)
            next_state, r, done,terminated, info = env.step(int(action))
            next_state=state_to_vector(next_state)
            # print("log_prob:", log_prob)
            # print("state:", torch.tensor(state).unsqueeze(0).float().to(device))
            # print("action:", torch.tensor(action).unsqueeze(0).unsqueeze(0))
            # print("next_state:", torch.tensor(next_state).unsqueeze(0).float().to(device))
            # print("done:", torch.tensor(done).view(1,1))


            reward = discriminator.get_reward(\
                                        log_prob,\
                                        torch.tensor(state).unsqueeze(0).float().to(device),torch.tensor(action).unsqueeze(0).unsqueeze(0),\
                                        torch.tensor(next_state).unsqueeze(0).float().to(device),\
                                                              torch.tensor(done).view(1,1)\
                                                 ).item()

            transition = make_transition(state,\
                                         torch.tensor(action),\
                                         np.array([reward/10.0]),\
                                         next_state,\
                                         np.array([done]),\
                                         log_prob.detach().cpu().numpy()\
                                        )
            agent.put_data(transition) 
            score += r

            discriminator_score += reward
            if done or terminated:
                state,_ = (env.reset())
                print(state)
                state=(state_to_vector(state))
                score_lst.append(score)
                if writer != None:
                    writer.add_scalar("score/real", score, n_epi)
                    writer.add_scalar("score/discriminator", discriminator_score, n_epi)
                score = 0
                discriminator_score = 0
            else:
                state = next_state
        state_lst = []
        
        if n_epi%args.print_interval==0 and n_epi!=0:
            if len(score_lst)==0:
                avg_score=0
            else:
                avg_score=sum(score_lst)/len(score_lst)
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, avg_score))
            score_lst = []
        print(discriminator_args.batch_size
        agent.train(discriminator, discriminator_args.batch_size, state_rms, n_epi)
        if (n_epi % args.save_interval == 0 )& (n_epi != 0):
                torch.save(agent.state_dict(), './model_weights/model_'+str(n_epi))
        if (n_epi % args.save_interval == 0 )& (n_epi != 0):
                torch.save(discriminator.state_dict(), './model_weights/discriminator_'+str(n_epi))



142
epi 0
0 traj
1 traj
2 traj
3 traj
4 traj
5 traj
6 traj
7 traj
8 traj
9 traj
10 traj
11 traj
12 traj
13 traj
14 traj
15 traj
16 traj
17 traj
18 traj
19 traj
20 traj
21 traj
22 traj
23 traj
24 traj
25 traj
26 traj
27 traj
28 traj
29 traj
30 traj
31 traj
32 traj
33 traj
34 traj
35 traj
36 traj
37 traj
38 traj
39 traj
40 traj
41 traj
42 traj
43 traj
44 traj
45 traj
46 traj
47 traj
48 traj
49 traj
50 traj
51 traj
52 traj
53 traj
54 traj
55 traj
56 traj
57 traj
58 traj
59 traj
60 traj
61 traj
62 traj
63 traj
64 traj
65 traj
66 traj
67 traj
68 traj
69 traj
70 traj
71 traj
72 traj
73 traj
74 traj
75 traj
76 traj
77 traj
78 traj
79 traj
80 traj
81 traj
82 traj
83 traj
84 traj
85 traj
86 traj
87 traj
88 traj
89 traj
90 traj
91 traj
92 traj
93 traj
94 traj
95 traj
96 traj
97 traj
98 traj
99 traj
100 traj
101 traj
102 traj
103 traj
104 traj
105 traj
106 traj
107 traj
108 traj
109 traj
110 traj
111 traj
112 traj
113 traj
114 traj
115 traj
116 traj
117 traj
118 traj
119 traj
120 traj
121 traj
12

/var/folders/30/6_15k8xn4kn7vwrk6x8py9sc0000gq/T/ipykernel_31161/3634274734.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(state).unsqueeze(0).float().to(device),torch.tensor(action).unsqueeze(0).unsqueeze(0),\
/var/folders/30/6_15k8xn4kn7vwrk6x8py9sc0000gq/T/ipykernel_31161/3634274734.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(next_state).unsqueeze(0).float().to(device),\


594 traj
595 traj
596 traj
597 traj
598 traj
599 traj
183
600 traj
601 traj
602 traj
603 traj
604 traj
605 traj
606 traj
607 traj
608 traj
609 traj
610 traj
611 traj
612 traj
613 traj
614 traj
615 traj
616 traj
617 traj
618 traj
619 traj
620 traj
621 traj
622 traj
623 traj
624 traj
625 traj
626 traj
627 traj
628 traj
629 traj
630 traj
631 traj
632 traj
633 traj
634 traj
635 traj
636 traj
637 traj
638 traj
639 traj
640 traj
641 traj
642 traj
643 traj
644 traj
645 traj
646 traj
647 traj
648 traj
649 traj
650 traj
651 traj
652 traj
653 traj
654 traj
655 traj
656 traj
657 traj
658 traj
659 traj
660 traj
661 traj
662 traj
663 traj
664 traj
665 traj
666 traj
667 traj
668 traj
669 traj
670 traj
671 traj
672 traj
673 traj
674 traj
675 traj
676 traj
677 traj
678 traj
679 traj
680 traj
681 traj
682 traj
683 traj
684 traj
685 traj
686 traj
687 traj
688 traj
689 traj
690 traj
691 traj
692 traj
693 traj
694 traj
695 traj
696 traj
697 traj
698 traj
699 traj
700 traj
701 traj
702 traj
703 traj
704 tr

KeyboardInterrupt: 